# Mask and No Mask Detection

### Importing the required libraries

In [1]:
import numpy as np
import cv2

#### Loading the file with_mask.npy and without_mask.npy

In [2]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [3]:
#checking the dimension of the with_mask images
with_mask.shape

(200, 50, 50, 3)

In [4]:
#checking the dimension of the without_mask images
without_mask.shape

(200, 50, 50, 3)

In [5]:
#converting 4 dimensional data into two dimensional
#both with and without mask contains 200 images with 7500 columns
with_mask = with_mask.reshape(200,50 * 50 * 3)
without_mask = without_mask.reshape(200, 50 * 50 * 3)

In [6]:
#again checking the dimension of with and without mask after reshaping them
with_mask.shape

(200, 7500)

In [7]:
without_mask.shape

(200, 7500)

In [8]:
#combining the data
#r_ in NumPy is used for concatination the rows of the data
X= np.r_[with_mask, without_mask]

In [9]:
#checking the shape of the X it is done for preparing the training data
X.shape

(400, 7500)

In [10]:
#labels is for y since we have total of 400 images
labels = np.zeros(X.shape[0])

In [11]:
#first 200 will point to with_mask images and 1.0 will point to without_mask images
labels[200:] = 1.0

if wearing mask the output will be 0 and if not wearing a mask output will be 1

In [54]:
names = {0 : 'Mask', 1: 'No Mask'}

# Machine Learning

It is the combiantion of program and maths.It is a type of artificial intelligence that allows software applications to become more accurate at predicting outcomes without being explicitly programmed to do so.

## Scikit Learn

 It is one of the library machine learinig in pyhton used for predicitive data analysis

### Support Vectore Machine

It is a set of supervised learning method used for detecting outliers, classification and regression detection. It is used because it is effective in high dimensional spaces implementation as well as it is useful in data where number of dimensions is greater than the number of samples.

In [12]:
 #svm - support Vector Machine
 #svc - support vector classification
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score # to check the accuracy score

In [13]:
#for dividing the data into training and testing split
from sklearn.model_selection import train_test_split

Here,training part will be used to apply the machine learning and testing part is used to perform and check the predictions, outcomes of the trained models.Also we are using 20 percent of data will be reserved for testing and the remaining 80 percent of data will be

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X,labels, test_size=0.25)

Since we have divided the data into training and testing machine learning will be applied on 300 images not on 400 images. On the reamining 100 images we will test the algorithm. 

In [15]:
x_train.shape

(300, 7500)

As we can notice that we have 7500 columns which is a problem resulting the program into slowing down the machine learning algorithm so now we can reduce the dimension using dimensionality reduction inorder to solve this problem.

#### Principal Component Analysis

It is a dimensionality-reduction method used to often reduce the dimensionality of a large data sets by transforming a large set of variables into a smaller one which contains most of the information in the large set.

##### Dimesionality Reduction
It is the transformation of data from high dimensional space into loe dimensional space so that the low dimensional ones contains some meaningful properties of the original data, close to intrinsic dimesion.

In [16]:
#Dimensionality reduction
#PCA - principal Component Analysis
from sklearn.decomposition import PCA

In [17]:
pca = PCA(n_components=3)#it will make the data into 3 columns
x_train = pca.fit_transform(x_train)

Dimentionality reduction uses Eigen Value and Eigen Vector to reduce the dimensions

### Eigen Value and Eigen Vectors

Eigen vectors are those unique vectors that do not chnage their direction when a linear transformation is performed on them. Whereas, Eigen value is the corresponding value number by which it is stretched or compressed due to linear transformation. Eigen valus is denoted by λ. It is calculated as Av-λv=0 where A is a square matrix and v is non zero vector and λ is eigen value.

It is calculated as follows:
For 2 dimension:
A=np.array([[5,3 ],[4,2]])
eigen_values,eigen_vectors=la.eig(A)
Ans: eigen values : [ 7.27491722 -0.27491722]
eigen vectors : 
 [[ 0.79681209 -0.49436913]
 [ 0.60422718  0.86925207]]

For 3 dimesion:
A=np.array([[-6, 3,4],[4,5,6],[2,7,4]])
la.eig(A)
eigen_values,eigen_vectors=la.eig(A)
Ans: eigen values:  [12.17784334 -6.88932678 -2.28851657]
eigen vectors:  [[-0.26169477 -0.93968193  0.19988471]
 [-0.70067295  0.3390297  -0.68579189]
 [-0.66375693 -0.0453512   0.69981111]]

In [18]:
x_train[0]

array([2340.25679839,  -84.74290094,  -93.93557389])

In [19]:
x_train.shape

(300, 3)

In [52]:
#shuffling the data
x_train, x_test, y_train, y_test = train_test_split(X,labels, test_size=0.20)

In [43]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [44]:
#x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)

Since the accuracy score was 1.0 in the beginning due to overfitting so we had to reshuffled the data until we got accurracy score we wanted.

In [53]:
#checking the actual test data and the prediction data
accuracy_score(y_test, y_pred)

0.4875

### Face Detector starting the camera

In [59]:
haar_data = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y), (x+w, y+h), (255,0,255), 4)
            #fetcing face of the image
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1, -1)
            #face = pca.transform(face)
            pred = svm.predict(face)
            n = names[int(pred)]
            cv2.putText(img, n, (x,y), font, 1, (244,250,250), 2)
            print(n)
        cv2.imshow('result',img)
        #27 is the ASCII value of escape
        if cv2.waitKey(2) == 27:
            break
            
capture.release()
cv2.destroyAllWindows()

No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mas